In [97]:
# Merge 12 files
#### function name: merge_stu_log(files_path) , input: files path list, output: DataFrame of all data
import pandas as pd
import numpy as np

def merge_stu_log(files_path):
    # Initial DataFrame
    df = pd.DataFrame()
    log1 = pd.read_csv(files_path[0])
    initial_col_order = [col.lower() for col in list(log1.columns)]
    # in loop
    for file in files_path:
        #load csv file as pandas
        student_log = pd.read_csv(file)
        student_log.columns = [col.lower() for col in student_log]
        df = pd.concat([df, student_log])

    df = df[initial_col_order]
    return df
student_log_list = ["../dataset/anonymized_dataset_for_ADM2017/student_log_{}.csv".format(i) for i in range(1,13)]
student_log = merge_stu_log(student_log_list)

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


# student list and question list

In [3]:
# student list
student_list = student_log['itest_id'].unique()
print(len(student_list))

# question list 
question_list = student_log['problemid'].unique()
print(len(question_list))

1709
4117


# students information

In [253]:
stu_information_list = ['itest_id','aveknow','avecarelessness','avecorrect',
                        'numactions','averesbored','averesengcon',
                         'averesconf','averesfrust','averesofftask','averesgaming']
student_information = student_log.loc[:,stu_information_list].drop_duplicates()  
student_information.to_csv("../dataset/student_information.csv",index=False)

# question information

In [254]:
# here don't use problemType because of inconsistency, might need cleaning
que_information_list = ['problemid','skill']

que_information = student_log.loc[:,que_information_list].drop_duplicates()
que_information_copy = que_information.copy()

p = que_information.groupby(['problemid']).count()
p = p[p['skill']>1]
for pr in p.index:
    a = que_information[que_information['problemid']==pr]
    b = a[a['skill']=='noskill']
    que_information.drop(b.index,inplace=True)
    
reduced_set = set(question_list) - set(que_information['problemid'])
for redu in reduced_set:
    que_information = pd.concat([que_information,que_information_copy[que_information_copy['problemid']==redu]])

que_information.to_csv("../dataset/question_information.csv",index=False)    
    

# Matrixs by different features

In [116]:
# pick up columns that should be used to filter
columns_list = ['itest_id','actionid','problemid','timetaken','correct',
                'hint','bottomhint','frishelprequest','scaffold',
                'frWorkingInSchool'.lower(),'responseIsFillIn'.lower(),
                'frIsHelpRequestScaffolding'.lower(),'timeOver80'.lower()]
students_actions = student_log.loc[:,columns_list]
group = students_actions.groupby(['itest_id','problemid'])

### action counting

In [117]:
action_count = group.count().loc[:,['actionid']]
action_count = action_count.rename(columns={'actionid':'action_count'})

### sum other features

In [122]:
sum_table = group.sum()
# the times of wrong action  = totalaction - action_request_hint - correct
sum_table['wrong'] = action_count['action_count'] - sum_table['hint'] - sum_table['correct']
sum_table['wrongrate'] = sum_table['wrong']/action_count['action_count']

# use hint rate
sum_table['hintrate'] = sum_table['hint']/action_count['action_count']

# correct could be more than once so the rate might more suitable
sum_table['correctrate'] = sum_table['correct']/action_count['action_count']

# first request help rate
sum_table['frhrate'] = sum_table['frishelprequest']/action_count['action_count']

# working in school rate (could be 1 or 0 in most of time)
sum_table['workinschoolrate'] = sum_table['frworkinginschool']/action_count['action_count']

# filedin (could be 1 or 0 in most of time)
sum_table['filedin'] = sum_table['responseisfillin']/action_count['action_count']

# first response is request scaffold help   (could be 1 or 0 in most of time)
sum_table['frscaffoldh'] = sum_table['frishelprequestscaffolding']/action_count['action_count']

# timeover80
sum_table['timeover80rate'] = sum_table['timeover80']/action_count['action_count']

sum_table.drop(['frishelprequest','frworkinginschool','responseisfillin',
                'frishelprequestscaffolding'],axis=1,inplace=True)
sum_table.drop(['actionid'],axis=1,inplace=True)

In [127]:
sum_table['actioncount'] = action_count['action_count']
matrix = sum_table.reindex(sorted(sum_table.columns), axis=1)

In [128]:
matrix.to_csv("../dataset/person_and_question_relationship.csv")